In [191]:
puzzle_input = open("puzzle16.txt").read().strip()

In [20]:
def hexToBin(hexadecimal : str) -> str:
    conv = {
        "0" : "0000", "1" : "0001", "2" : "0010", "3" : "0011",
        "4" : "0100", "5" : "0101", "6" : "0110", "7" : "0111",
        "8" : "1000", "9" : "1001", "A" : "1010", "B" : "1011",
        "C" : "1100", "D" : "1101", "E" : "1110", "F" : "1111",
    }
    return "".join([conv[c]for c in hexadecimal])

In [58]:
def getHeader(packet: str) -> tuple[int, int]:
    'Returns version number, type id'
    ver = packet[:3]
    tid = packet[3:6]
    return int(ver, 2), int(tid, 2)

In [86]:
def getLenTID(packet : str) -> tuple[int, int, int]:
    '''Returns length type ID, length segment size, length value'''
    conv = {"0": 15, "1": 11}
    if getHeader(packet)[1] == 4:
        raise ValueError("LITERAL VALUE PACKET DOES NOT HAVE LENGTH TYPE ID.")
    else:
        subHLen = conv[packet[6]]
        subHVal = int(packet[7:7 + subHLen], 2)
        return int(packet[6]), subHLen, subHVal

In [156]:
def getLitValue(packet : str) -> tuple[int, int, int]:
    '''Returns literal value, payload length, total length'''
    head = getHeader(packet)
    if head[1] != 4: raise ValueError("PACKET IS NOT A LITERAL VALUE.")
    binv = ""
    cursor,groups = 6, 0
    while True:
        binv += packet[cursor + 1 : cursor + 5]
        groups += 1
        if packet[cursor] == "0": break
        cursor += 5
    l = len(binv) + groups
    return int(binv, 2), l, l + 6

In [210]:
def bundlePacket(data : str, maxlen : int = -1) -> dict():
    ver, tid = getHeader(data)
    bundle = {"ver": ver,"tid":tid}
    if tid == 4:
        litv,litl,totl = getLitValue(data)
        bundle["size"] = totl
        bundle["literal"] = {"val" : litv,"len" : litl}
        bundle["bits"] = data[0:totl]
        return bundle
    ltid,lenbits,lenv = getLenTID(data)
    dh = 7 + lenbits
    bundle["ltid"]=ltid
    bundle["body"] = {"subpackets":[]}
    if ltid == 0:
        bundle["size"] = dh + lenv
        bundle["bits"] = data[0:dh + lenv]
        bundle["body"]["len"] = lenv
        bundle["body"]["data"] = data[dh:dh + lenv]
    else:
        bundle["bits"] = data
        bundle["body"]["count"] = lenv
        bundle["body"]["rest"] = data[dh:]
        if maxlen >= 0: bundle["size"] = maxlen
    return bundle
    

In [211]:
ese2 = bundlePacket("00111000000000000110111101000101001010010001001000000000")
ese3 = bundlePacket("11101110000000001101010000001100100000100011000001100000")

ese2, ese3

({'ver': 1,
  'tid': 6,
  'ltid': 0,
  'body': {'subpackets': [], 'len': 27, 'data': '110100010100101001000100100'},
  'size': 49,
  'bits': '0011100000000000011011110100010100101001000100100'},
 {'ver': 7,
  'tid': 3,
  'ltid': 1,
  'body': {'subpackets': [],
   'count': 3,
   'rest': '01010000001100100000100011000001100000'},
  'bits': '11101110000000001101010000001100100000100011000001100000'})

In [213]:
p = ese3
clen, count, cursor = 0,0,0
lt0 = p["ltid"] == 0
lt1 = p["ltid"] == 1
subpackets = []

if lt0:
    clen = 0
    while clen < p["body"]["len"]:
        pckt = bundlePacket(p["body"]["data"][clen:])
        clen += pckt["size"]
        p["body"]["subpackets"].append(pckt)
elif lt1:
    cursor = 0
    for _ in range(p["body"]["count"]):
        pckt = bundlePacket(p["body"]["rest"][cursor:])
        count += 1
        cursor += pckt["size"]
        p["body"]["subpackets"].append(pckt)

p

{'ver': 7,
 'tid': 3,
 'ltid': 1,
 'body': {'subpackets': [{'ver': 2,
    'tid': 4,
    'size': 11,
    'literal': {'val': 1, 'len': 5},
    'bits': '01010000001'},
   {'ver': 4,
    'tid': 4,
    'size': 11,
    'literal': {'val': 2, 'len': 5},
    'bits': '10010000010'},
   {'ver': 1,
    'tid': 4,
    'size': 11,
    'literal': {'val': 3, 'len': 5},
    'bits': '00110000011'}],
  'count': 3,
  'rest': '01010000001100100000100011000001100000'},
 'bits': '11101110000000001101010000001100100000100011000001100000'}

In [224]:
def extractSubpackets(packet : str, maxPacketLength : int = -1):
    print("Extracting packet...")
    p = bundlePacket(packet, maxPacketLength)
    lt0 = p["ltid"] == 0
    lt1 = p["ltid"] == 1
    if lt0:
        print("Type: 0")
        clen = 0
        while clen < p["body"]["len"]:
            pckt = bundlePacket(p["body"]["data"][clen:])
            if getHeader(pckt["bits"])[1] != 4:
                pckt = extractSubpackets(pckt["bits"], p["body"]["len"])
                if getLenTID(pckt["bits"])[0] == 1:
                    pckt["size"] = sum([ sp["size"] for sp in pckt["body"]["subpackets"]])
            clen += pckt["size"]
            p["body"]["subpackets"].append(pckt)
    elif lt1:
        print("Type: 1")
        cursor = 0
        for _ in range(p["body"]["count"]):
            pckt = bundlePacket(p["body"]["rest"][cursor:])
            print(pckt["bits"][1])
            if getHeader(pckt["bits"])[1] != 4:
                pckt = extractSubpackets(pckt["bits"])
                if getLenTID(pckt["bits"])[0] == 1:
                    pckt["size"] = sum([ sp["size"] for sp in pckt["body"]["subpackets"]])
            cursor += pckt["size"]
            p["body"]["subpackets"].append(pckt)
    return p

In [225]:
extractSubpackets(hexToBin(puzzle_input))

Extracting packet...
Type: 1
1
Extracting packet...
Type: 0
1
Extracting packet...
Type: 1
0
0
1
0
Extracting packet...
Type: 1
0
1
Extracting packet...
Type: 0
Extracting packet...
Type: 0
1
Extracting packet...
Type: 1
0
Extracting packet...
Type: 0
Extracting packet...
Type: 0
Extracting packet...
Type: 0
1
0
Extracting packet...
Type: 0
Extracting packet...
Type: 1
0
Extracting packet...
Type: 0
Extracting packet...
Type: 1
0
Extracting packet...
Type: 0
Extracting packet...
Type: 1
0
Extracting packet...
Type: 0
Extracting packet...
Type: 0
Extracting packet...
Type: 1
0
1
Extracting packet...
Type: 1
0
Extracting packet...
Type: 0
Extracting packet...
Type: 1
0
Extracting packet...
Type: 0
Extracting packet...
Type: 1
1
Extracting packet...
Type: 0
Extracting packet...
Type: 0
Extracting packet...
Type: 1
1
Extracting packet...
Type: 1
0
Extracting packet...
Type: 0
Extracting packet...
Type: 1
0
Extracting packet...
Type: 0
Extracting packet...
Type: 0
Extracting packet...
Type:

IndexError: string index out of range